Keras Tuner with GRU and LSTM layers

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 7.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
import keras_tuner as kt
df = pd.read_csv("drive/MyDrive/Engineer's Project/test.csv")

In [ ]:
df.pop('usa_cpi')
df.pop('pol_cpi')
df.pop('usa_inter')
df.pop('pol_inter')
df.pop('Date')

0       2010.11.15
1       2010.11.16
2       2010.11.17
3       2010.11.18
4       2010.11.19
           ...    
3537    2022.03.27
3538    2022.03.28
3539    2022.03.29
3540    2022.03.30
3541    2022.03.31
Name: Date, Length: 3542, dtype: object

In [ ]:
scaler = MinMaxScaler()
df_numpy = df.to_numpy() 
#len(df_numpy[:2500])
scaler1 = scaler.fit(df_numpy[:3000])
df_scalled = scaler1.transform(df_numpy)
#df_scaled_all = scaler.
df_scalled = pd.DataFrame(df_scalled, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
lookback = 15
step = 1
delay = 1
batch_size = 128

In [ ]:
float_data = np.array(df_scalled).astype('float32')

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] 
    yield samples, targets

In [ ]:
train_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=2501,
#shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
val_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=2501,
max_index=3001,
step=step,
batch_size=batch_size)

In [ ]:
test_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=3001,
max_index=3543,
step=step,
batch_size=batch_size)

In [ ]:
train_steps = (2501 - lookback)
val_steps = (3001 - 2501 - lookback)
test_steps = (3543 - 3001 - lookback)

In [ ]:
def model_builder(hp):
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  hp_LSTMunits = hp.Choice('LSTM neurons', values=[10,50,100,200,300])
  hp_GRUunits = hp.Choice('GRU neurons', values=[1,5,10,20,40])
  #hp_Denseunits = hp.Int('Dense neurons', min_value=1, max_value=32, step=1)
  model = tf.keras.models.Sequential()
  model.add(layers.LSTM(units=hp_LSTMunits,
    return_sequences=True, 
    activation = hp.Choice('LSTM activation',values=['sigmoid','softmax']),
    recurrent_dropout=0.1,
    #dropout=0.15,
    input_shape=(None, float_data.shape[-1])))
  model.add(layers.Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
  model.add(layers.GRU(units=hp_GRUunits,
    activation=hp.Choice('GRU activation',values=['sigmoid','softmax']),
    #activation='sigmoid',
    recurrent_dropout=0.1,
    #dropout=0.15,
    input_shape=(None, float_data.shape[-1])))
  model.add(layers.Dropout(hp.Float('Dropout_rate2',min_value=0,max_value=0.5,step=0.1)))
  model.add(layers.Dense(units=5,
    #activation=hp.Choice('Dense activation',values=['relu', 'sigmoid','softmax'])))
    activation='sigmoid'))
  model.add(layers.Dense(1,
    activation='sigmoid'))
    #activation = hp.Choice('Dense last activation',values=['relu', 'sigmoid','softmax'])))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate), loss='mae')
  return model

In [ ]:
def model_builder5(hp):
  #hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  hp_LSTMunits = hp.Int('LSTM neurons', min_value=112, max_value=196, step=16)
  #hp_GRUunits = hp.Choice('GRU neurons', values=[20,40,60])
  hp_Denseunits = hp.Int('Dense neurons', min_value=0, max_value=128, step=16)
  #hp_Denseunits = hp.Int('Dense neurons', min_value=1, max_value=32, step=1)
  model = tf.keras.models.Sequential()
  model.add(layers.LSTM(units=hp_LSTMunits,
    #return_sequences=True, 
    #activation = hp.Choice('LSTM activation',values=['sigmoid','softmax']),
    activation='sigmoid',
    recurrent_dropout=0.1,
    #dropout=0.5,
    input_shape=(None, float_data.shape[-1])))
  model.add(layers.Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
  #model.add(layers.GRU(units=hp_GRUunits,
    #activation=hp.Choice('GRU activation',values=['sigmoid','softmax']),
   # activation='sigmoid',
   # recurrent_dropout=0.1,
    #dropout=0.15,
   # input_shape=(None, float_data.shape[-1])))
  #model.add(layers.Dropout(hp.Float('Dropout_rate2',min_value=0,max_value=0.5,step=0.1)))
  model.add(layers.Dense(units=hp_Denseunits,
    #activation=hp.Choice('Dense activation',values=['relu', 'sigmoid','softmax'])))
    activation='sigmoid'))
  model.add(layers.Dense(1,
    #activation='sigmoid'))
    activation = hp.Choice('Dense last activation',values=['relu', 'sigmoid','softmax'])))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.01), loss='mae')
  return model

In [ ]:
tunerBayesian = kt.BayesianOptimization(model_builder,
    objective='val_loss',
    max_trials=100,
    )


In [ ]:
tunerHyper = kt.Hyperband(model_builder5,
    objective='val_loss',
    directory='hyper')

In [ ]:
tunerHyper.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 147 Complete [00h 35m 24s]
val_loss: 0.009093498811125755

Best val_loss So Far: 0.009093498811125755
Total elapsed time: 03h 57m 15s

Search: Running Trial #148

Value             |Best Value So Far |Hyperparameter
144               |176               |LSTM neurons
128               |80                |Dense neurons
0.1               |0.1               |Dropout_rate
sigmoid           |sigmoid           |Dense last activation
100               |100               |tuner/epochs
34                |34                |tuner/initial_epoch
4                 |4                 |tuner/bracket
4                 |4                 |tuner/round
0142              |0145              |tuner/trial_id

Epoch 35/100
200/200 [==============================] - 21s 96ms/step - loss: 0.1533 - val_loss: 0.0378
Epoch 36/100
117/200 [================>.............] - ETA: 5s - loss: 0.0630

KeyboardInterrupt: ignored

In [ ]:
tunerHyper.save()

In [ ]:
!cp -r "hyper/untitled_project/" "drive/MyDrive/Colab Notebooks"

In [ ]:
tunerBayesian3 = kt.BayesianOptimization(model_builder2,
    objective='val_loss',
    max_trials=9,
    directory='third'
    )


In [ ]:
tunerBayesian4 = kt.BayesianOptimization(model_builder3,
    objective='val_loss',
    max_trials=15,
    directory='4'
    )

In [ ]:
tunerBayesian4.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 11 Complete [00h 13m 26s]
val_loss: 0.03083822876214981

Best val_loss So Far: 0.017995454370975494
Total elapsed time: 02h 22m 15s

Search: Running Trial #12

Value             |Best Value So Far |Hyperparameter
300               |300               |LSTM neurons
60                |60                |GRU neurons
40                |10                |Dense neurons

Epoch 1/10
200/200 [==============================] - 84s 398ms/step - loss: 0.1656 - val_loss: 0.0380
Epoch 2/10
200/200 [==============================] - 77s 387ms/step - loss: 0.1530 - val_loss: 0.1776
Epoch 3/10
200/200 [==============================] - 77s 386ms/step - loss: 0.1162 - val_loss: 0.0511
Epoch 4/10
200/200 [==============================] - 77s 386ms/step - loss: 0.1164 - val_loss: 0.2407
Epoch 5/10
200/200 [==============================] - 79s 396ms/step - loss: 0.1100 - val_loss: 0.0736
Epoch 6/10
200/200 [==============================] - 77s 387ms/step - loss: 0.1115 - val_loss: 0.2367
Epoch 7/1

KeyboardInterrupt: ignored

In [ ]:
tunerHyper.results_summary()

Results summary
Results in hyper/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
LSTM neurons: 176
Dense neurons: 80
Dropout_rate: 0.1
Dense last activation: sigmoid
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 4
tuner/round: 4
tuner/trial_id: 0145
Score: 0.009093498811125755
Trial summary
Hyperparameters:
LSTM neurons: 176
Dense neurons: 80
Dropout_rate: 0.1
Dense last activation: sigmoid
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 4
tuner/round: 3
tuner/trial_id: 0137
Score: 0.009271706454455853
Trial summary
Hyperparameters:
LSTM neurons: 144
Dense neurons: 128
Dropout_rate: 0.1
Dense last activation: sigmoid
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 4
tuner/round: 3
tuner/trial_id: 0132
Score: 0.009396069683134556
Trial summary
Hyperparameters:
LSTM neurons: 144
Dense neurons: 128
Dropout_rate: 0.1
Dense last activation: sigmoid
tuner/epochs: 12
tuner/initial_epoch: 4
tuner/bracket: 4
tuner/round: 2
tuner/trial_id: 0112
S

In [ ]:
tunerBayesian.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 100 Complete [00h 01m 37s]
val_loss: 0.015227541327476501

Best val_loss So Far: 0.01089098397642374
Total elapsed time: 06h 20m 55s
INFO:tensorflow:Oracle triggered exit


In [ ]:
params = tunerBayesian.get_best_hyperparameters()
print(params)
models = tunerBayesian.get_best_models()

In [ ]:
#print(params[0])
tunerBayesian.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 300
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.5
GRU activation: sigmoid
Dropout_rate2: 0.0
Score: 0.01089098397642374
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.011510745622217655
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.01157438475638628
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.011914853937923908
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activatio

Results summary
Results in ./untitled_project
Showing 10 best trials
<keras_tuner.engine.objective.Objective object at 0x7fd35f868b10>
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 300
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.5
GRU activation: sigmoid
Dropout_rate2: 0.0
Score: 0.01089098397642374
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.011510745622217655
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.01157438475638628
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.011914853937923908
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.012222672812640667
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.012414476834237576
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 100
GRU neurons: 10
LSTM activation: sigmoid
Dropout_rate: 0.5
GRU activation: softmax
Dropout_rate2: 0.0
Score: 0.012458978220820427
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.012510736472904682
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.012713002972304821
Trial summary
Hyperparameters:
learning_rate: 0.01
LSTM neurons: 10
GRU neurons: 40
LSTM activation: sigmoid
Dropout_rate: 0.0
GRU activation: sigmoid
Dropout_rate2: 0.1
Score: 0.013040080666542053